# Correlation of PPE Demand in USA With Covid19 Cases

In [ ]:
# import data
import json
import time
import requests
from io import StringIO
import os

# computing
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

# Import geopandas package
import geopandas as gpd
import reverse_geocoder as rg
import addfips
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.io
from plotly.offline import plot
import us

# plotting
import plotly.express as px
import plotly.graph_objects as go

## Configs

In [ ]:
findthemasks_url = 'http://findthemasks.com/data.json'
request_headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3"}
county_fips_download_url = 'https://github.com/ShyamW/Geocoding_Suite/blob/master/Lat_Lng_to_County_Data/county_Fips.txt'
geojson_url = 'https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json'
ny_times_covid_date = '2020-03-30'
ny_times_county_data_url = 'https://github.com/nytimes/covid-19-data/raw/master/us-counties.csv'
# Import hospital information compiled by https://beta.covidmap.link/
hospital_download_url = 'https://docs.google.com/spreadsheet/ccc?key=15gZsozGQp-wdJaSngvLV13iCf_2mm2IsZpHOPxZtvtI&output=csv'

## Download find the mask data and convert to pandas
- Taken from find the mask [web visualization](https://findthemasks.com/give.html) 
- [Data updated every 5 mins here](findthemasks.com/data.json) - The data visulized here is from 3/25 at 10PM PST

In [ ]:
def download_findthemasks_data(url,request_headers):
    # Download the data
    s=requests.get(url, headers= request_headers).text

    # Extract the json format, and find column headers
    json_data = json.loads(s)
    HEADERS = json_data['values'][0]

    # create the data frame
    mask_df = pd.DataFrame.from_dict(json_data['values'][2:])
    mask_df.columns=HEADERS
    
    # Using DataFrame.drop
    mask_df = mask_df.dropna(how='any', subset=['Lat', 'Lng'])

    # Rename the State? column
    mask_df.rename(columns={'State?': 'State'}, inplace=True)

    # Drop institutions with multiple entries
    mask_df.drop_duplicates(subset='What is the name of the hospital or clinic?', inplace=True)

    return mask_df


mask_df = download_findthemasks_data(url = findthemasks_url, request_headers = request_headers)
mask_df.head(10000)

### Create geocoder class to find fips and county information by lat/long

In [ ]:
class geocoder:
    def __init__(self, county_fips_download_url):
        self.af = addfips.AddFIPS()
        self.download_county_fips_info(county_fips_download_url)
        
    def download_county_fips_info(self, url):
        contents=requests.get(url).text
        with open('county_Fips.txt', 'w') as f:
            f.write(contents)    
        
    def fips_code_lookup(self, county, state):
        # Lookup of fips code (https://github.com/fitnr/addfips)
        fips = self.af.get_county_fips(county, state)
        return fips

    def get_geocoder_info_from_rg(self, Lat, Lng):
        try:
            # Reverse geocoder api call to get county name
            coordinates = (Lat, Lng)
            results = rg.search(coordinates) # default mode = 2
            county = results[0]['admin2']
            state = results[0]['admin1']

            # Lookup of fips code (https://github.com/fitnr/addfips)
            fips = self.fips_code_lookup(county,state)

            # return the fip and county
            return {'fips':fips, 'county':county}
        except ValueError:
            return {'fips':'NA', 'county':'NA'}

### Search and add the FIPS code to each row - WILL TAKE SEVERAL MINS

In [ ]:
def add_fips_county_info(mask_df, geocoder):
    # Start tdqm timer from tqdm.auto
    tqdm.pandas()

    # Reverse geocoder used to get geocoded fips and county information
    # Note: Progress_apply is used for the timer functionality
    mask_df['geocoder'] = mask_df.progress_apply(
        lambda x: geocoder.get_geocoder_info_from_rg(x['Lat'], x['Lng']), axis=1)

    # Map the geocoder dict column to individual columns
    mask_df['fips'] = mask_df.apply(
        lambda x: x['geocoder']['fips'], axis=1)
    mask_df['county'] = mask_df.apply(
        lambda x: x['geocoder']['county'], axis=1)
    mask_df.drop(columns=['geocoder'],inplace = True)

    # Using DataFrame.drop to remove any fips code that could not be mapped
    mask_df = mask_df.dropna(how='any', subset=['fips','county'])
    
    return mask_df

geocoder = geocoder(county_fips_download_url)
mask_df = add_fips_county_info(mask_df, geocoder)

### Sum amount of requests per county

In [ ]:
def requests_per_county(mask_df, write_out_csv = True):
    # Count the amount of requests per county
    mask_df_counties=mask_df.groupby(['fips','county','State']).size().reset_index(name='counts')
    
    # write out this data file to csv
    if write_out_csv:
        timestr = time.strftime("%Y%m%d")
        path = 'findthemasks_data_processed_' + timestr + '.csv'
        mask_df.to_csv (path, index = False, header=True)

        ##### TODO
        # Some of the data written out is corrupted and misaligned by row
        # Not sure what the bug is right now
    
    return mask_df_counties


mask_df_counties = requests_per_county(mask_df, write_out_csv = True)
mask_df_counties.head(5)

### Download county geo information

In [ ]:
def download_county_geojson(geojson_url):
    # Download the data
    s=requests.get(geojson_url).text

    # Extract the json format, and find column headers
    counties = json.loads(s)
    
    # Create counties_df from geojson counties object
    counties_df = pd.DataFrame.from_dict(counties['features'])
    counties_df['properties'][0]

    # extract properties dict, then concatenate new clumsn and remove old properties column
    counties_df = pd.concat(
        [counties_df, pd.json_normalize(counties_df['properties'])], axis=1).drop(['properties'], axis=1)

    # clean up the dataframe                                                                               
    counties_df.drop(['type','COUNTY','LSAD'], axis=1, inplace=True)
    counties_df.rename(columns={'id':'fips','NAME':'county'}, inplace=True)
    counties_df.head()
    
    # join with the dataframe that has ppe requests: mask_df
    merged_df = counties_df.join(
        mask_df_counties[['fips','counts']].set_index('fips'),
        on='fips',  how='left', lsuffix='counties', rsuffix='mask_df')

    # fill the NA in counts with 0s
    merged_df['counts'].fillna(0, inplace=True)
    
    # change name of column 'counts' to 'PPE_requests' 
    merged_df.rename(inplace=True,
        columns={'counts':'PPE_requests'})
    
    # Map fips state code to state name
    merged_df['STATE'] = merged_df.progress_apply(
        lambda x: us.states.lookup(x['STATE']), axis=1)
    merged_df['county_info_for_map'] = merged_df.progress_apply(
        lambda x: ('PPE Requests: %s, %s'%(x['county'],x['STATE'])), axis=1)
    
    # Create text column for use in mapping
    merged_df['ppe_text'] = 'PPE Requests: ' + merged_df['PPE_requests'].astype(int).astype(str) + '<br>'+ \
        merged_df['county'].astype(str) + ', ' + merged_df['STATE'].astype(str)


    
    # return a json object called counties for plotting, and a counties_df for joins+manipulation of other data
    return counties, merged_df


counties, merged_df = download_county_geojson(geojson_url)
merged_df.head(2)

### Function for mapping requests by County

In [ ]:
def choropleth_mapbox_usa_plot (counties, locations, z, text,
                                colorscale = "RdBu_r", zmin=-1, zmax=10, 
                                title='choropleth_mapbox_usa_plot',
                                colorbar_title = 'count',
                                html_filename='plot.html'):
    
    # Choropleth graph. For reference: https://plotly.com/python/mapbox-county-choropleth/
    fig = go.Figure(go.Choroplethmapbox(
        geojson=counties, locations=locations, z=z, text=text,
        colorscale=colorscale,zmin=zmin,zmax=zmax,marker_opacity=0.8, 
        marker_line_width=0.5, colorbar_title=colorbar_title, hoverinfo='text'
        ))
    
    # Center on US
    fig.update_layout(
        title=title,
        mapbox_style="carto-positron",
        mapbox_zoom=3.5, 
        mapbox_center = {"lat": 37.0902, "lon": -95.7129},
        margin={"r":300,"t":30,"l":30,"b":0},
    )
    # Download the figure
    plot(fig, filename=html_filename)

### Map of PPE Requests

In [ ]:
choropleth_mapbox_usa_plot(
    counties = counties,
    locations = merged_df.fips,
    z = merged_df.PPE_requests,
    text = merged_df.ppe_text,
    #colorscale = ["#fdfcef", "#c7e9b4", "#6ab7a6","#41b6c4","#2c7fb8","#253494"],
    colorscale = ["#fdfcef","#c7e9b4","#D2FBFF","#36A2B9","#004469"],
    zmin = 0,
    zmax=5,
    title = ('PPE Requests By County - %s - (Hover for breakdown)' % time.strftime("%Y%m%d")),
    colorbar_title = '> PPE Requests',
    html_filename = 'PPE_Requests_By_County.html')

## Download COVID19 data and convert to pandas

In [ ]:
def download_findthemasks_data(url, date, write_out_csv = True):
    covid_df = pd.read_csv(url)
    covid_df = covid_df.loc[covid_df['date'] == date]

    # NYC data is missing county, so make them all New York County.
    covid_df.loc[covid_df['county'] == 'New York City', 'fips'] = '36061'
    # Kansas City data is missing the specific county so make them all Cook County
    covid_df.loc[(covid_df['county'] == 'Kansas City') & 
              (covid_df['state'] == 'Missouri'), 'fips'] = '29095'
    
    # drop the rows without a fips value
    covid_df = covid_df.dropna(how='any', subset=['fips'])

    # convert to int to remove the decimal values
    covid_df['fips'] = covid_df['fips'].apply(int)
    
    # Zfill all countyFIPS to be 5 characters
    width=5
    covid_df["fips"]= covid_df["fips"].astype(str)
    covid_df["fips"]= covid_df["fips"].str.zfill(width) 
        
    # write out this data file to csv
    if write_out_csv:
        timestr = time.strftime("%Y%m%d")
        path = 'COVID19_nytimes_' + date + ' data_processed_on_' + timestr + '.csv'
        covid_df.to_csv (path, index = False, header=True)

    return covid_df
    

covid_df = download_findthemasks_data(ny_times_county_data_url, ny_times_covid_date, write_out_csv = True)
covid_df.head(5)

In [ ]:
def merge_covid_ppe_df(covid_df,merged_df): 
    merged_covid_ppe_df = merged_df.join(
        covid_df[['fips','cases','deaths']].set_index('fips'),
        on='fips',  how='left', lsuffix='merged', rsuffix='covid_df')
    
    # fill the NA in counts with 0s
    merged_covid_ppe_df['cases'].fillna(0, inplace=True)
    merged_covid_ppe_df['deaths'].fillna(0, inplace=True)
    
    # Create text column for use in mapping
    merged_covid_ppe_df['covid_text'] = merged_covid_ppe_df['county'].astype(str) + ', ' + \
        merged_covid_ppe_df['STATE'].astype(str) + '<br><br>'+\
        'Covid19: ' + '<br>'+ \
        'Cases: ' + merged_covid_ppe_df['cases'].astype(int).astype(str) + '<br>'+ \
        'Deaths: ' + merged_covid_ppe_df['deaths'].astype(int).astype(str) + '<br><br>'+ \
        'PPE Requests: ' + merged_covid_ppe_df['PPE_requests'].astype(int).astype(str)
        
    return merged_covid_ppe_df


merged_covid_ppe_df=merge_covid_ppe_df(covid_df,merged_df) 
merged_covid_ppe_df.head(5)

##### Merge the counties geojson for all of new york

In [ ]:
counties['features'][0]

In [ ]:
''' 
from shapely.geometry import Polygon
from shapely.ops import cascaded_union

polygon1 = Polygon([(0, 0), (5, 3), (5, 0)])
polygon2 = Polygon([(0, 0), (3, 10), (3, 0)])

polygons = [polygon1, polygon2]

u = cascaded_union(polygons)
'''

### Mapping covid cases 

In [ ]:
choropleth_mapbox_usa_plot(
    counties = counties,
    locations = merged_covid_ppe_df.fips,
    z = merged_covid_ppe_df.cases,
    text = merged_covid_ppe_df.covid_text,
    colorscale = ["#fdfcef","#ffda55","#FFC831","#fc7555","#e96e81",],
    zmin = 0,
    zmax=100,
    title = ('COVID19 Cases Per County - %s - (Hover for breakdown)' % ny_times_covid_date),
    html_filename = ('COVID19_Cases_Per_County_%s.html' % ny_times_covid_date),
    colorbar_title = '> COVID19 Cases',
)

## Hospital bed visualization by county 

In [ ]:
def download_hospital_data(url, write_out_csv = True):
    hospital_df = pd.read_csv(hospital_download_url)
    # Start tdqm timer from tqdm.auto
    tqdm.pandas()

    # Reverse geocoder used to get geocoded fips and county information
    # Note: Progress_apply is used for the timer functionality
    hospital_df['fips'] = hospital_df.progress_apply(
        lambda x: geocoder.fips_code_lookup(x['COUNTY'], x['STATE']), axis=1)
    
    # clean the BEDS column to make sure all are positive in value, by converting negative beds to 0
    hospital_df.sort_values(by=['BEDS'], ascending=False, inplace=True)
    hospital_df['BEDS'][hospital_df['BEDS'] < 0] = 0
    
    # write out this data file to csv
    if write_out_csv:
        timestr = time.strftime("%Y%m%d")
        path = 'hospital_data_processed_' + timestr + '.csv'
        mask_df.to_csv (path, index = False, header=True)

    return hospital_df


hospital_df = download_hospital_data(hospital_download_url, write_out_csv = True)
hospital_df.head(3)

In [ ]:
def process_hospital_data(hospital_df, write_out_csv = True):
    # Sum the amount of beds per county
    hospital_df_counties = hospital_df.groupby(['fips','COUNTY'])['BEDS'].sum().reset_index()

    # write out this data file to csv
    if write_out_csv:
        timestr = time.strftime("%Y%m%d")
        path = 'hospital_data_county_data_' + timestr + '.csv'
        mask_df.to_csv (path, index = False, header=True)
        
    return hospital_df_counties
    
    
hospital_df_counties = process_hospital_data(hospital_df, write_out_csv = True)
hospital_df_counties.head(2)

In [ ]:
def merge_covid_ppe_hosp_df(hospital_df_counties,merged_covid_ppe_df): 
    merged_covid_ppe_hosp_df = merged_covid_ppe_df.join(
        hospital_df_counties[['fips','BEDS']].set_index('fips'),
        on='fips',  how='left', lsuffix='merged', rsuffix='hospital')
    
    # fill the NA in counts with 0s
    merged_covid_ppe_hosp_df['BEDS'].fillna(0, inplace=True)
    
    # Create text column for use in mapping
    merged_covid_ppe_hosp_df['hosp_text'] = merged_covid_ppe_hosp_df['county'].astype(str) + ', ' + \
        merged_covid_ppe_hosp_df['STATE'].astype(str) + '<br><br>'+\
        'Hospital Beds: ' + merged_covid_ppe_hosp_df['BEDS'].astype(int).astype(str) + '<br>'+ \
        '<br>'+ \
        'Covid19: ' + '<br>'+ \
        'Cases: ' + merged_covid_ppe_hosp_df['cases'].astype(int).astype(str) + '<br>'+ \
        'Deaths: ' + merged_covid_ppe_hosp_df['deaths'].astype(int).astype(str) + '<br><br>'+ \
        'PPE Requests: ' + merged_covid_ppe_hosp_df['PPE_requests'].astype(int).astype(str)
        
    return merged_covid_ppe_hosp_df


merged_covid_ppe_hosp_df=merge_covid_ppe_hosp_df(hospital_df_counties,merged_covid_ppe_df) 
merged_covid_ppe_hosp_df.head(5)

In [ ]:
# Hospital bed plotting
choropleth_mapbox_usa_plot(
    counties = counties,
    locations = merged_covid_ppe_hosp_df.fips,
    z = merged_covid_ppe_hosp_df.BEDS,
    text = merged_covid_ppe_hosp_df.hosp_text,
    colorscale = ["#fdfcef","#c7e9b4","#D2FBFF","#36A2B9","#004469"],
    zmin = 0,
    zmax=500,
    title = ('Hospital beds per county - %s - (Hover for breakdown)' % time.strftime("%Y%m%d")),
    html_filename = ('Hospital_beds_per_county_%s.html' % time.strftime("%Y%m%d")),
    colorbar_title = '> Hospital Beds'
    )

### Covid cases per bed available

In [ ]:
# In order to avoid divide by zero problem in lambda
def weird_division(n, d):
    return n / d if d else 0

def calculate_covid_per_bed_available(merged_covid_ppe_hosp_df):
    # calculate the covid patients per bed, adding the column that saves this info
    merged_covid_ppe_hosp_df['Covid_cases_per_bed'] = merged_covid_ppe_hosp_df.apply(
            lambda x: (weird_division(x['cases'], x['BEDS'])), axis=1)
    
    # sort by highest normalized_covid_patients_per_bed
    merged_covid_ppe_hosp_df.sort_values(by='Covid_cases_per_bed', ascending=False, inplace=True)
    
    # Create text column for use in mapping
    merged_covid_ppe_hosp_df['hosp_text'] = merged_covid_ppe_hosp_df['county'].astype(str) + ', ' + \
        merged_covid_ppe_hosp_df['STATE'].astype(str) + '<br><br>'+ \
        'HAZARD RATIO (Cases/Bed): ' + merged_covid_ppe_hosp_df['Covid_cases_per_bed'].astype(float).astype(str) + '<br><br>'+ \
        'Hospital Beds: ' + merged_covid_ppe_hosp_df['BEDS'].astype(int).astype(str) + '<br>'+ \
        '<br>'+ \
        'Covid19: ' + '<br>'+ \
        'Cases: ' + merged_covid_ppe_hosp_df['cases'].astype(int).astype(str) + '<br>'+ \
        'Deaths: ' + merged_covid_ppe_hosp_df['deaths'].astype(int).astype(str) + '<br><br>'+ \
        'PPE Requests: ' + merged_covid_ppe_hosp_df['PPE_requests'].astype(int).astype(str)
    
    return merged_covid_ppe_hosp_df


merged_covid_ppe_hosp_df = calculate_covid_per_bed_available(merged_covid_ppe_hosp_df)
merged_covid_ppe_hosp_df.head(5)

In [ ]:
# Map hazard ratio
choropleth_mapbox_usa_plot(
    counties = counties,
    locations = merged_covid_ppe_hosp_df.fips,
    z = merged_covid_ppe_hosp_df.Covid_cases_per_bed,
    text = merged_covid_ppe_hosp_df.hosp_text,
    colorscale = ["#fdfcef","#ffda55","#FFC831","#fc7555","#e96e81",],
    zmin = 0,
    zmax=1,
    title = ('Hazard Ratio: Covid19 Cases, Per Bed, Per County - %s - (Hover for breakdown)' % ny_times_covid_date),
    html_filename = ('Covid19_cases_per_bed_per_county_%s.html' % time.strftime("%Y%m%d")),
    colorbar_title = '> Hazard Ratio (Cases/Bed)'
    )

### Counties without PPE requests, with highest Covid19 cases

In [ ]:
def find_counties_with_covid19_and_no_ppe_request(covid_df, mask_df_counties):
    # join the covid patients dataframe with the beds per county dataframe, on the fips index
    covid_ppe_df = covid_df.join(
        mask_df_counties.set_index('fips'), on='fips',  how='left', lsuffix='_covid', rsuffix='_ppe')
    
    # fill the NA in normalized_covid_patients_per_bedwith 0s
    covid_ppe_df['counts'].fillna(0, inplace=True)
    
    # sort by highest normalized_covid_patients_per_bed
    covid_ppe_df.sort_values(by=['counts','cases'], ascending=(True, False), inplace=True)
    
    # change name of column 'counts' to 'PPE_requests' 
    covid_ppe_df.rename(inplace=True,
        columns={'counts':'PPE_requests', 'county_covid':'county'})
    
    ### TODO
    # There may be a mismatch of the PPE requests lat/long and those of the hospital data
    # since District of Columbia is appearing at the top, and that is unlikely
    
    return covid_ppe_df


covid_ppe_df = find_counties_with_covid19_and_no_ppe_request(covid_df, mask_df_counties)
covid_ppe_df[['date','county','state','cases','deaths','PPE_requests']].head(15)

In [ ]:
# Select the counties that have no_ppe_requests and covid cases
counties_with_no_ppe_requests_and_covid_cases = covid_ppe_df[covid_ppe_df.PPE_requests == 0]

# Map covid cases in counties that do not have PPE requests
choropleth_mapbox_usa_plot(
    counties = counties,
    locations = counties_with_no_ppe_requests_and_covid_cases.fips,
    z = counties_with_no_ppe_requests_and_covid_cases.cases,
    text = counties_with_no_ppe_requests_and_covid_cases.county,
    colorscale = "RdBu_r",
    zmin = 0,
    zmax=100,
    title = 'Counties that have covid cases and 0 PPE requests'
    )

### Correlation of PPE request per county with COVID19 cases

In [ ]:
# select counties that have had at least 1 ppe request
counties_with_ppe_requests_and_covid_cases = covid_ppe_df[covid_ppe_df.PPE_requests != 0]

# join with the dataframe that has covid cases per hospital bed
covid_ppe_df = counties_with_ppe_requests_and_covid_cases.join(
    covid_per_bed_df[['county','state','fips','Covid_cases_per_bed','BEDS',]].set_index('fips'),
    on='fips',  how='left', lsuffix='', rsuffix='_ppe')

# sort by highest normalized_covid_patients_per_bed
counties_with_ppe_requests_and_covid_cases.sort_values(by=['PPE_requests','cases'], ascending=False, inplace=True)
counties_with_ppe_requests_and_covid_cases[
    ['date','county','state','cases','deaths','BEDS','PPE_requests','Covid_cases_per_bed']].head(20)

In [ ]:
fig = px.scatter(
    counties_with_ppe_requests_and_covid_cases,
    x=counties_with_ppe_requests_and_covid_cases.cases, 
    y=counties_with_ppe_requests_and_covid_cases.PPE_requests,
    color='Covid_cases_per_bed',
    log_x=True,
    #log_y=True,
    labels={
        'Covid_cases_per_bed':'Covid19 cases per hospital bed',
        'x':'Covid19 Cases Per County',
        'y':'PPE Requests Per County',
        'text':'County'
        },
    hover_name=counties_with_ppe_requests_and_covid_cases.county,
    range_color=(0,1),
    range_x=(1,30000)
    )

fig.update_layout(
    title = "Correlation of PPE request per county with COVID19 cases",
    #hoverlabel={'text'},
    )

#fig.update_xaxes(nticks=30)
#fig.update_yaxes(nticks=20)
    
fig.show()

In [ ]:
<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://getusppe.github.io/PPE_Requests_Per_County/.embed" height="525" width="100%"></iframe>